In [4]:
%load_ext jupyter_black

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.collections import LineCollection
from matplotlib.lines import Line2D
from matplotlib import rc

rc("text", usetex=True)
rc("text.latex", preamble=r"\usepackage{xcolor}")

from copy import deepcopy
from subprocess import Popen
from IPython.display import IFrame
from pathlib import Path
from itertools import chain

from TestSet import read_Chao, read_Tsiligirides, TestInstance

In [6]:
Tsiligirides = read_Tsiligirides()
Chao = read_Chao()

FileNotFoundError: 

In [7]:
sum([len(Tsiligirides[i]) for i in Tsiligirides] + [len(Chao[i]) for i in Chao])

NameError: name 'Tsiligirides' is not defined

# TOP helper functions

In [5]:
def route_cost(cost_matrix: np.ndarray, route: list[int]):
    """Compute the cost of a route given a cost matrix."""
    return cost_matrix[route[:-1], route[1:]].sum()

In [6]:
def route_reward(profit_vector: np.ndarray, route: list[int]):
    """Compute the reward of a route given a profit vector."""
    return profit_vector[route].sum()

In [7]:
def route_improved_cost_profit(
    cost_matrix: np.ndarray,
    profit_vector: np.ndarray,
    old_route: list[int],
    new_route: list[int],
):
    """Return if a route has improved."""
    old_cost = route_cost(cost_matrix, old_route)
    new_cost = route_cost(cost_matrix, new_route)
    old_reward = route_reward(profit_vector, old_route)
    new_reward = route_reward(profit_vector, new_route)
    return (
        # Improvement in reward (opt. cost)
        (old_cost <= new_cost and old_reward > new_reward)
        or
        # Improvement in cost (opt. reward)
        (old_cost < new_cost and old_reward >= new_reward)
    )

In [8]:
def next_feasible_stop(instance: TestInstance, route: list[int]) -> list:
    """Return feasible stops from a given route last stop.

    Args:
        - instance: TestInstance
        - route: list of indexes

    Stops are feasible if:
    - route cost + stop cost <= T_max, and
    """
    if not type(instance) is TestInstance:
        raise TypeError

    s = route[-1]

    # Check if route has ended.
    if s == instance.N - 1:
        return []

    # Route has not ended, continue.
    r_cost = route_cost(instance.C, route)
    feasible_mask = (
        # Add costs source + depot + route cost
        instance.C[s] + instance.C[-1] + r_cost
        <= instance.tmax
    )
    feasible_stops = np.argsort(np.where(feasible_mask, instance.C[route[-1]], np.inf))
    return np.setdiff1d(  # Remove routed stops
        np.setdiff1d(  # Remove unfeasible stops
            feasible_stops,
            (~feasible_mask).nonzero(),
            assume_unique=True,  # No sorting
        ),
        instance.routed.nonzero(),
        assume_unique=True,  # No sorting
    )

In [9]:
def cost_change(cost_mat, n1, n2, n3, n4):
    return cost_mat[n1][n3] + cost_mat[n2][n4] - cost_mat[n1][n2] - cost_mat[n3][n4]


def two_opt(cost_matrix: np.ndarray, route: list[int]):
    """Improve a route using the two-opt edge exchange method.

    Source: https://stackoverflow.com/questions/53275314/2-opt-algorithm-to-solve-the-travelling-salesman-problem-in-python
    """
    best = route
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route)):
                if j - i == 1:
                    continue
                if (
                    cost_change(cost_matrix, best[i - 1], best[i], best[j - 1], best[j])
                    < 0
                ):
                    best[i:j] = best[j - 1 : i - 1 : -1]
                    improved = True
        route = best
    return best

In [10]:
def reverse_segment_if_better(cost_matrix: np.ndarray, route: list[int], i: int):
    """If reversing route[i:j] would make the route shorter, then do it.

    Args:
        - route: list of nodes
        - i: starting node
    """
    # Given route [...A-B-C-D...]
    i, j, k = i, i + 1, i + 2
    A, B, C, D = route[i - 1], route[i], route[j], route[k]
    d0 = cost_matrix[A, B] + cost_matrix[B, C] + cost_matrix[C, D]
    d1 = cost_matrix[A, B] + cost_matrix[B, C] + cost_matrix[C, D]
    d2 = cost_matrix[A, B] + cost_matrix[B, C] + cost_matrix[C, D]
    d3 = cost_matrix[A, C] + cost_matrix[C, B] + cost_matrix[B, D]
    d4 = cost_matrix[D, B] + cost_matrix[B, C] + cost_matrix[C, A]

    if d0 > d1:
        route[i:j] = reversed(route[i:j])
        return -d0 + d1
    elif d0 > d2:
        route[j:k] = reversed(route[j:k])
        return -d0 + d2
    elif d0 > d4:
        route[i:k] = reversed(route[i:k])
        return -d0 + d4
    elif d0 > d3:
        tmp = route[j:k] + route[i:j]
        route[i:k] = tmp
        return -d0 + d3
    return 0

In [11]:
def all_consecutive_segments(n: int):
    """Generate all consecutive segments combinations"""
    return ((i, i + 1, i + 2) for i in range(n - 2))

In [12]:
def or_opt(cost_matrix: np.ndarray, route: list[int]):
    """Improve a route using the Or-opt (restricted 3-opt) exchange method.

    Source: https://en.wikipedia.org/wiki/3-opt
    """
    while True:
        delta = 0
        for a, _, _ in all_consecutive_segments(len(route)):
            delta += reverse_segment_if_better(cost_matrix, route, a)
        if delta >= 0:
            break
    return route

In [13]:
def plot_routes(
    instance: TestInstance,
    routes: dict[int, list[int]],
    colours: list[str] = ["red", "blue", "green", "black"],
    figsize=(10, 10),
    edge_labels=False,
):
    """Plot a single route on the instance graph."""
    fig, ax = plt.subplots(figsize=figsize)
    G = nx.DiGraph()
    G.add_nodes_from(
        (x, {"weight": p}) for x, p in zip(list(range(instance.N)), instance.P)
    )
    nodes = list(G)
    routed_nodes = set(chain(*solution.solution.values()))
    routed_nodes.remove(nodes[0])  # Remove start node
    routed_nodes.remove(nodes[-1])  # Remove terminal node
    unrouted_nodes = set(nodes) - routed_nodes

    _ = nx.draw_networkx_nodes(
        G,
        nodelist=routed_nodes,
        pos=instance.X,
        node_size=200,
        node_color="k",
        node_shape="o",
        ax=ax,
    )
    _ = nx.draw_networkx_nodes(
        G,
        nodelist=unrouted_nodes,
        pos=instance.X,
        node_size=200,
        node_color="g",
        node_shape="d",
        ax=ax,
    )
    _ = nx.draw_networkx_nodes(
        G,
        nodelist=[nodes[0], nodes[-1]],
        pos=instance.X,
        node_size=300,
        node_color="r",
        node_shape="s",
        ax=ax,
    )
    _ = nx.draw_networkx_labels(
        G,
        pos=instance.X,
        font_color="w",
        ax=ax,
    )
    title = (
        f"Instance: {instance.name}, "
        f"Best solution: {instance.best_solution:.0f}, "
        f"Total profit: {np.sum(instance.P):.0f}"
        f"\n"
    )
    total_reward = 0
    total_cost = 0
    for (route_idx, route), colour in zip(routes.items(), colours):
        G = nx.create_empty_copy(G)  # copy without edges
        G.add_weighted_edges_from(
            (u, v, c)
            for u, v, c in zip(route[:-1], route[1:], instance.C[route[:-1], route[1:]])
        )
        _ = nx.draw_networkx_edges(G, pos=instance.X, edge_color=colour)
        if edge_labels:
            _ = nx.draw_networkx_edge_labels(
                G,
                pos=instance.X,
                edge_labels={
                    (i[0], i[1]): f'{i[2]["weight"]:.1f}' for i in G.edges(data=True)
                },
            )
        reward = route_reward(instance.P, route)
        cost = route_cost(instance.C, route)
        total_reward += reward
        total_cost += cost
        title += (
            r"\textcolor{" + colour + r"}{(Route n°" + f"{route_idx:02d}" + r")} "
            # f"(Route n°{route_idx:02d}) "
            f"Reward: {reward:.2f}, "
            f"Cost: {cost:.2f}, "
            r"$T_{\max}$ : " + f"{instance.tmax:.2f}\n"
        )

    _ = plt.title(
        title + f"Total reward: {total_reward:.2f}, Total Cost: {total_cost:.2f}\n"
    )

    return fig, ax

In [14]:
def plot_to_pdf(
    name: Path,
    instance: TestInstance,
    routes: dict[int, list[int]],
    colours: list[str] = ["red", "blue", "green", "black"],
    figsize=(8.5, 10),
):
    """Plot routes and export figure to pdf."""
    if not name.parent.exists():
        raise ValueError("Parent directory does not exist")

    fig, ax = plot_routes(instance, solution.solution, colours, figsize)
    name = name.with_name("alns_" + name.name)
    plt.tight_layout()
    plt.savefig(str(name))
    plt.close()
    Popen("ps2pdf " + str(name), shell=True, cwd=str(name.parent)).wait()
    return IFrame(str(name.with_suffix(".pdf")), width=800, height=600)

# Augmented Large Neighbourhood Search

## Pseudo-Code

Parameters:
- Max iterations $I_\max$
- Number of solutions $N$

1. Construct an initial solution $s^*$ (**Algorithm 1**)
2. Add $s^*$ to the solution pool $S$ 
3. Until $I_\max$
    1. Destroy routes (**Algorithm 2**)
    2. Repair routes (**Algorithm 3**)
    3. Insert maximum number of unrouted stops (**Algorithm 4**)
    4. Improve the solution until no further improvement (**Algorithm 5**)
    5. (Terminate if the solution is the upper bound, for existing optimal solutions) (**Algorithm D**)
    6. Update the best solution $S^*$ if there is improvement
    7. Replace the worst solution $S$ if
        1. The current solution is not in the pool
        2. The pool is full
        3. The pool's worst solution is worse than the current solution
    8. Insert the new solution in $S$ if
        1. $S$ is not full
        2. The current solution is not in $S$

# Algorithm A: Local Search Improvement
> [The algorithm] consists of popular inter-route and intraroute improvement methods. In the 1-1 improvement method, a stop from a route is exchanged with a stop from another route. The exchange is accepted if it is feasible for both routes and the total travel distance of the two routes is reduced. In the 1-0 improvement method, a stop from a route is moved into another route and improvement is tested. Similarly, the 2-1 improvement method exchanges two stops from a route with one stop from another route. The intra-route improvement algorithm consists of well-known 2opt edge exchange method and Or-opt improvement method (Or, 1976). The improvement methods repeat until no further improvement is possible.

In [15]:
def intra_route_improvement(cost_matrix: np.ndarray, solution: dict[int, list[int]]):
    """2-opt + Or-opt."""
    improved = True
    best = deepcopy(solution)
    while improved:
        improved = False
        for team in range(len(solution)):
            # Apply 2-opt, then Or-opt
            new_route = or_opt(cost_matrix, two_opt(cost_matrix, best[team]))

            # Check for improvement in terms of cost
            if route_cost(cost_matrix, new_route) < route_cost(cost_matrix, best[team]):
                improved = True
                best[team] = new_route

    return best

# Algorithm B: Insertion Method
> In the insertion method, each unrouted stop is tested to be inserted into a route and the unrouted feasible stop that has the smallest distance increase due to the stop insertion is iteratively selected and inserted. The insertion procedure is repeated until no additional stops can be inserted to any of the routes

In [16]:
def best_insertion(
    cost_matrix: np.ndarray,
    tmax: float,
    routed: list[int],
    solution: dict[int, list[int]],
) -> tuple[dict[int, list[int]], list[int]]:
    """Iteratively insert as many as possible unrouted stops iteratively.

    Returns:
        - new soution
        - new routed index
    """
    unrouted = (~routed).nonzero()[0]
    insertion_possible = np.ones((len(solution), len(unrouted)), dtype=bool)
    new_solution = deepcopy(solution)
    new_routed = deepcopy(routed)

    # Repeat until no stops can be inserted anymore
    while np.sometrue(insertion_possible):
        for team in range(len(new_solution)):
            # For each route in the solution
            route = new_solution[team]
            r_cost = route_cost(cost_matrix, route)
            # Find the best feasible stop where to insert the unrouted stop
            costs = np.array(
                [
                    [
                        r_cost
                        - cost_matrix[u, v]
                        + cost_matrix[[u, unrouted_stop], [unrouted_stop, v]].sum()
                        for u, v in zip(route[:-1], route[1:])
                    ]
                    for unrouted_stop in unrouted
                ]
            )

            # Check feasibility
            feasible_mask = (
                # End stop is already included
                costs
                <= tmax
            )
            for i in range(feasible_mask.shape[1]):
                feasible_mask[:, i] = np.logical_and(
                    feasible_mask[:, i],
                    insertion_possible[team],
                )

            # Mark unfeasible stops
            # (if an unrouted stop cannot be inserted anywhere in the route)
            insertion_possible[team, feasible_mask.sum(axis=1) == 0] = False

            if np.sometrue(feasible_mask):
                costs = np.where(feasible_mask, costs, np.inf)
                best = np.unravel_index(costs.argmin(), costs.shape)

                # Insert best stop into route
                new_stop = unrouted[best[0]]
                route.insert(best[1] + 1, new_stop)
                new_routed[new_stop] = True

                # Unrouted stop cannot be inserted anymore
                insertion_possible[:, best[0]] = False

    return new_solution, new_routed

# Algorithm C: Shifting and Insertion Method
> [A]ttempts are made to move stops from a route into other routes to make room for unrouted stops. [...] [A]ll possible unrouted stops are inserted into a route; [Then, the algorithm attempts to add as many unrouted stops as possible to the route].

# Algorithm D1: Random Replacement Method
> The algorithm exchanges routed stops with unrouted stops to improve the solution’s total reward value. [...] [A] random number is generated for each iteration and this number determines the number of stops that will be removed from the route. [...] [The] stops to be deleted are selected randomly and their total rewards are compared with the total rewards of all unrouted stops. [...] [If] the value of the unrouted stops is less than the value of the deleted stops, that iteration will be skipped because there is no potential benefit to the exchange. Otherwise [...] the selected stops are deleted from the route and unrouted stops are randomly inserted. In this step, each unrouted stop is tested to determine if the insertion is feasible. [...] [If] the total rewards of the newly inserted stops are larger than the total rewards of the deleted stops, the replacement for the route will be accepted and the next route will be tested. The above procedures repeat for $R_\max$ iterations for each route.

# Algorithm D2: Full-Enumeration Replacement Method
> Instead of randomly selecting $d_r$ routed stops and randomly replacing current stops, the full-enumeration replacement algorithm tests all routed stops for a 1-1 replacement and then a 1-2 replacement. The 1-1 replacement exchanges a routed stop with an unrouted stop and repeats until there is no improvement. Then 1-2 replacement is tested checking the replacement of a routed stop with two unrouted stops. The replacement is executed when the solution rewards are increased or the route travel time is decreased without rewards loses.

# Algorithm E: Known upper bound termination
> [The] algorithm terminates if the improved solution is the upper bound, where the upper bound terminate condition is applied for those instances in which the exact optimal solutions are reported by Boussier, Feillet, and Gendreau (2007). [...] Note that Dang et al. (2011) also used the same upper bound terminate condition.

# Algorithm 1: Greedy Construction
> From the current stop, $c_L$, the feasible stop that has the smallest distance divided by reward is selected as the next stop. The procedure repeats until no more stops can be added to the route. [...] When there is improvement, i.e., the total route travel distance is reduced, additional stops are inserted. [...] [T]he algorithm attempts to add as many unrouted stops as possible to the route [...].

In [17]:
def construction(inst: TestInstance) -> TestInstance:
    # Initialise instance
    instance = deepcopy(inst)
    instance.routed = np.zeros(instance.N - 1, dtype=bool)  # Stop is never routed
    instance.routed[0] = True  # Start is always routed
    instance.solution = {team: [0] for team in range(instance.m)}

    # For each team
    for team in range(instance.m):
        # Repeat while there is a feasible unrouted stop
        while (
            len(feasible_list := next_feasible_stop(instance, instance.solution[team]))
            > 0
        ):
            # Greedy feasible stop
            c = feasible_list[0]

            # Add feasible stop with lowest distance to the route
            instance.solution[team] += [c]

            # Set c_N routed
            if c != instance.N - 1:
                instance.routed[c] = True

        # Improve routes using the intra-route improvement
        instance.solution[team] = intra_route_improvement(
            instance.C, {0: instance.solution[team]}
        )[0]

        # Insertion method
        instance.solution, instance.routed = best_insertion(
            instance.C, instance.tmax, instance.routed, instance.solution
        )

    return instance

In [18]:
# Instance
instance = Chao["Set_100_234"][1]

# Set all stops unrouted
solution = construction(instance)

NameError: name 'Chao' is not defined

# Algorithm 2: Destroy routes
> [...] $k$ stops are removed from the selected solution according to the remove criterion. Three remove criterions are designed for solution diversification: random remove, biggest remove and smallest remove. In the random remove, $k$ stops will be randomly selected and removed from the current solution. The biggest remove will remove the $k$ stops which have the biggest rewards and the $k$ smallest rewards stops will be removed under the smallest remove criterion similarly. The three remove criterions are randomly selected in each iteration.

In [20]:
def random_remove(solution: dict[int, list[int]], k: int = 1) -> dict[int, list[int]]:
    """Destroy solution by removing k random stops."""
    new_solution = deepcopy(solution)
    team = np.random.randint(len(solution))
    remove = np.random.choice(len(new_solution[team]) - 2, k, replace=False)
    
    for stop in sorted(remove, reverse=True):
        new_solution[team].pop(stop)
        
    return new_solution, remove


def biggest_remove(profit_vector: np.ndarray, solution: dict[int, list[int]], k: int = 1) -> dict[int, list[int]]:
    """Destroy solution by removing the k stops which have the biggest rewards."""
    new_solution = deepcopy(solution)
    team = np.random.randint(len(solution))
    remove = np.argsort([route_reward(profit_vector, s) for s in new_solution[team][1:-1]])[-k:]
    
    for stop in sorted(remove, reverse=True):
        new_solution[team].pop(stop + 1)  
    
    return new_solution, remove

def smallest_remove(profit_vector: np.ndarray, solution: dict[int, list[int]], k: int = 1) -> dict[int, list[int]]:
    """Destroy solution by removing the k stops which have the smallest rewards."""
    new_solution = deepcopy(solution)
    team = np.random.randint(len(solution))
    remove = np.argsort([route_reward(profit_vector, s) for s in new_solution[team][1:-1]])[:k]
        
    for stop in sorted(remove, reverse=True):
        new_solution[team].pop(stop + 1)
        
    return new_solution, remove


# Algorithm 3: Repair routes
> [...] [The] solution with stops removed is repaired via two improvement methods until there are no more improvements [...].

The algorithms mentionned are the Local Search Improvement (A) and Shifting and Insertion methods (C)

# Algorithm 4: Maximum re-insertion
> In the insertion method, each unrouted stop is tested to be inserted into a route and the unrouted feasible stop that has the smallest distance increase due to the stop insertion is iteratively selected and inserted. The insertion procedure is repeated until no additional stops can be inserted to any of the routes.

# Algorithm 5: Maximum improvement
> [The] updated solution is further improved not only by the previous two improvement methods, but also with two replacement method [...].

The two replacement methods are the Random Replacement (D1) and the Full-Enumeration Replacement (D2) methods.